# I am choosing to use a random forest as my initial model, and trying minority class oversampling along with majority class undersampling

In [1]:
import pandas as pd
import numpy as np
import h2o
from h2o.estimators.random_forest import H2ORandomForestEstimator

C:\Users\Mikhail\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
h2o.init(nthreads = -1)

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 25.181-b13, mixed mode)
  Starting server from C:\Users\Mikhail\Anaconda\lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\Mikhail\AppData\Local\Temp\tmp2ug5jnl5
  JVM stdout: C:\Users\Mikhail\AppData\Local\Temp\tmp2ug5jnl5\h2o_Mikhail_started_from_python.out
  JVM stderr: C:\Users\Mikhail\AppData\Local\Temp\tmp2ug5jnl5\h2o_Mikhail_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,05 secs
H2O cluster timezone:,America/New_York
H2O data parsing timezone:,UTC
H2O cluster version:,3.28.0.3
H2O cluster version age:,1 month and 21 days
H2O cluster name:,H2O_from_python_Mikhail_dpokzt
H2O cluster total nodes:,1
H2O cluster free memory:,1.738 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"accepting new members, healthy"


In [3]:
data = pd.read_csv('C:\\Users\\Mikhail\\Documents\\Data 606\\cleaned_data_reduced_feats.csv')
h2o_data = h2o.import_file('C:\\Users\\Mikhail\\Documents\\Data 606\\cleaned_data_reduced_feats.csv')

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [4]:
#Making sure categorical columns are correctly encoded as such
cont_columns = ['Start_Lat', 'Start_Lng', 'Temperature(F)', 'Humidity(%)', 'Pressure(in)', 'Visibility(mi)', 'Wind_Speed(mph)']

for c in data.columns:
    if c not in cont_columns:
        h2o_data[c] = h2o_data[c].asfactor()
        data[c] = data[c].astype('category')

#For combined under/over random sampling
class_weights = [80.0, .3, .4, 40.0]

features = list(h2o_data.columns)
features.remove('Severity')
response = 'Severity'

In [5]:
#Baseline model run. Hyperparameters have not be optimized, want to get an initial feel for model performance.
h_train, h_valid, h_test = h2o_data.split_frame([0.6, 0.2], seed=1234)

rf_v1 = H2ORandomForestEstimator(model_id = 'rf_v1', balance_classes = True,
                                 class_sampling_factors = class_weights,
                                 ntrees = 50, max_depth = 10, categorical_encoding = 'enum')

In [6]:
rf_v1.train(x = features, y = response, training_frame = h_train, validation_frame = h_valid)

drf Model Build progress: |███████████████████████████████████████████████| 100%


In [7]:
rf_v1

Model Details
H2ORandomForestEstimator :  Distributed Random Forest
Model Key:  rf_v1


Model Summary: 


,,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
0,,50.0,200.0,918211.0,10.0,10.0,10.0,113.0,642.0,362.375




ModelMetricsMultinomial: drf
** Reported on train data. **

MSE: 0.38574380941099445
RMSE: 0.6210827717872993
LogLoss: 1.782149387863889
Mean Per-Class Error: 0.5341704471239918

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class


,1,2,3,4,Error,Rate
0,0.0,23521.0,559.0,0.0,1.000000,"24,080 / 24,080"
1,0.0,115400.0,6122.0,0.0,0.050378,"6,122 / 121,522"
2,0.0,6982.0,73918.0,0.0,0.086304,"6,982 / 80,900"
3,0.0,6828.0,81852.0,0.0,1.000000,"88,680 / 88,680"
4,0.0,152731.0,162451.0,0.0,0.399338,"125,864 / 315,182"



Top-4 Hit Ratios: 


...


Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_classification_error,validation_rmse,validation_logloss,validation_classification_error
0,,2020-03-26 22:02:58,0.044 sec,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,,2020-03-26 22:03:02,3.931 sec,1.0,0.640360,1.902479,0.437435,0.312608,0.392735,0.121214
2,,2020-03-26 22:03:10,11.230 sec,8.0,0.622927,1.784350,0.406766,0.247293,0.233203,0.068901
3,,2020-03-26 22:03:22,23.336 sec,21.0,0.621235,1.777918,0.400301,0.242269,0.228370,0.064892
4,,2020-03-26 22:03:48,49.681 sec,50.0,0.621083,1.782149,0.399338,0.241799,0.228877,0.064901



Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,Interstate_Hwy,1.428042e+06,1.000000,0.401748
1,road_blocked,3.596698e+05,0.251862,0.101185
2,State_Hwy,2.362643e+05,0.165446,0.066468
3,Hour,1.700524e+05,0.119081,0.047841
4,Side,1.644503e+05,0.115158,0.046264
5,Start_Lng,1.643989e+05,0.115122,0.046250
6,Pressure(in),1.422977e+05,0.099645,0.040032
7,Numbered_Hwy,1.326691e+05,0.092903,0.037324
8,Month,1.257900e+05,0.088086,0.035388
9,Traffic_Signal,1.201921e+05,0.084166,0.033813



See the whole table with table.as_data_frame()


Terrible results for a baseline run. The model, while it classies level 2 and 3 collisions very accurately, is completing ignoring the minority classes. There are several options on how to proceed from here. There are a number of very useless features measured by Gini in the RF model that may be producing inaccurate results, especially among trees in the forest that select only these. Further feature selection could be useful. More engineered features are a possibility, especially with the lack of an exact definition of "severity." Text analysis of the accident description may be key to both understanding how severe accidents differ from minor ones, determining whether turning this into a binary classification problem is appropriate, as well as providing opportunites to add more features. Hyperparameter tuning is an obvious neccessity, and different models can be experimented with as well. First, I am going to try synthetically generating more minority samples to see if this makes a difference. 